# SMS Spam Classification with MLflow

This notebook demonstrates experiment tracking and model version control using MLflow.
We'll build 3 benchmark models and track their performance using AUCPR (Area Under Precision-Recall Curve) as the primary metric.

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
    auc
)
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## 2. MLflow Setup

In [ ]:
# Set experiment name
EXPERIMENT_NAME = "SMS_Spam_Classification"
mlflow.set_experiment(EXPERIMENT_NAME)

# Get experiment info
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print(f"Experiment Name: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")
print(f"\nMLflow Tracking URI: {mlflow.get_tracking_uri()}")

## 3. Load Data

In [ ]:
# Load datasets
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('validation.csv')
test_df = pd.read_csv('test.csv')

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nClass distribution in training set:")
print(train_df['label'].value_counts())
print(f"\nSpam ratio: {(train_df['label'] == 'spam').sum() / len(train_df):.2%}")

## 4. Prepare Data for Training

In [ ]:
# Extract features and labels
X_train = train_df['text'].values
y_train = (train_df['label'] == 'spam').astype(int).values

X_val = val_df['text'].values
y_val = (val_df['label'] == 'spam').astype(int).values

X_test = test_df['text'].values
y_test = (test_df['label'] == 'spam').astype(int).values

print("Data prepared successfully!")
print(f"Training labels - Ham: {(y_train == 0).sum()}, Spam: {(y_train == 1).sum()}")
print(f"Validation labels - Ham: {(y_val == 0).sum()}, Spam: {(y_val == 1).sum()}")
print(f"Test labels - Ham: {(y_test == 0).sum()}, Spam: {(y_test == 1).sum()}")

## 5. Helper Functions for Evaluation

In [ ]:
def calculate_metrics(y_true, y_pred, y_pred_proba):
    """Calculate comprehensive metrics including AUCPR"""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'aucpr': average_precision_score(y_true, y_pred_proba)  # Area Under PR Curve
    }
    return metrics

def print_metrics(metrics, dataset_name=""):
    """Pretty print metrics"""
    print(f"\n{'='*50}")
    print(f"{dataset_name} Metrics")
    print(f"{'='*50}")
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1 Score:  {metrics['f1_score']:.4f}")
    print(f"ROC AUC:   {metrics['roc_auc']:.4f}")
    print(f"AUCPR:     {metrics['aucpr']:.4f}")
    print(f"{'='*50}")

print("Helper functions defined!")

## 6. Model 1: Logistic Regression

In [ ]:
# Start MLflow run
with mlflow.start_run(run_name="Logistic_Regression") as run:
    print(f"MLflow Run ID: {run.info.run_id}")
    
    # Model parameters
    params = {
        'model_type': 'Logistic Regression',
        'max_features': 5000,
        'ngram_range': '(1, 2)',
        'C': 1.0,
        'max_iter': 1000,
        'solver': 'liblinear'
    }
    
    # Log parameters
    mlflow.log_params(params)
    
    # Create pipeline
    model = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
        ('classifier', LogisticRegression(C=1.0, max_iter=1000, solver='liblinear', random_state=42))
    ])
    
    # Train model
    print("\nTraining Logistic Regression...")
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    
    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    
    y_test_pred = model.predict(X_test)
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    val_metrics = calculate_metrics(y_val, y_val_pred, y_val_proba)
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    
    # Log metrics to MLflow
    for metric_name, value in train_metrics.items():
        mlflow.log_metric(f"train_{metric_name}", value)
    for metric_name, value in val_metrics.items():
        mlflow.log_metric(f"val_{metric_name}", value)
    for metric_name, value in test_metrics.items():
        mlflow.log_metric(f"test_{metric_name}", value)
    
    # Print results
    print_metrics(train_metrics, "Training")
    print_metrics(val_metrics, "Validation")
    print_metrics(test_metrics, "Test")
    
    # Log model to MLflow
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="SMS_Spam_LogisticRegression"
    )
    
    print(f"\n✓ Model registered as 'SMS_Spam_LogisticRegression'")
    print(f"✓ Test AUCPR: {test_metrics['aucpr']:.4f}")
    
    lr_run_id = run.info.run_id
    lr_aucpr = test_metrics['aucpr']

## 7. Model 2: Multinomial Naive Bayes

In [ ]:
# Start MLflow run
with mlflow.start_run(run_name="Naive_Bayes") as run:
    print(f"MLflow Run ID: {run.info.run_id}")
    
    # Model parameters
    params = {
        'model_type': 'Multinomial Naive Bayes',
        'max_features': 5000,
        'ngram_range': '(1, 2)',
        'alpha': 1.0
    }
    
    # Log parameters
    mlflow.log_params(params)
    
    # Create pipeline
    model = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
        ('classifier', MultinomialNB(alpha=1.0))
    ])
    
    # Train model
    print("\nTraining Multinomial Naive Bayes...")
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    
    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    
    y_test_pred = model.predict(X_test)
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    val_metrics = calculate_metrics(y_val, y_val_pred, y_val_proba)
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    
    # Log metrics to MLflow
    for metric_name, value in train_metrics.items():
        mlflow.log_metric(f"train_{metric_name}", value)
    for metric_name, value in val_metrics.items():
        mlflow.log_metric(f"val_{metric_name}", value)
    for metric_name, value in test_metrics.items():
        mlflow.log_metric(f"test_{metric_name}", value)
    
    # Print results
    print_metrics(train_metrics, "Training")
    print_metrics(val_metrics, "Validation")
    print_metrics(test_metrics, "Test")
    
    # Log model to MLflow
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="SMS_Spam_NaiveBayes"
    )
    
    print(f"\n✓ Model registered as 'SMS_Spam_NaiveBayes'")
    print(f"✓ Test AUCPR: {test_metrics['aucpr']:.4f}")
    
    nb_run_id = run.info.run_id
    nb_aucpr = test_metrics['aucpr']

## 8. Model 3: Random Forest

In [ ]:
# Start MLflow run
with mlflow.start_run(run_name="Random_Forest") as run:
    print(f"MLflow Run ID: {run.info.run_id}")
    
    # Model parameters
    params = {
        'model_type': 'Random Forest',
        'max_features': 5000,
        'ngram_range': '(1, 2)',
        'n_estimators': 100,
        'max_depth': 20,
        'min_samples_split': 5,
        'min_samples_leaf': 2
    }
    
    # Log parameters
    mlflow.log_params(params)
    
    # Create pipeline
    model = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
        ('classifier', RandomForestClassifier(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        ))
    ])
    
    # Train model
    print("\nTraining Random Forest...")
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    
    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    
    y_test_pred = model.predict(X_test)
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    val_metrics = calculate_metrics(y_val, y_val_pred, y_val_proba)
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    
    # Log metrics to MLflow
    for metric_name, value in train_metrics.items():
        mlflow.log_metric(f"train_{metric_name}", value)
    for metric_name, value in val_metrics.items():
        mlflow.log_metric(f"val_{metric_name}", value)
    for metric_name, value in test_metrics.items():
        mlflow.log_metric(f"test_{metric_name}", value)
    
    # Print results
    print_metrics(train_metrics, "Training")
    print_metrics(val_metrics, "Validation")
    print_metrics(test_metrics, "Test")
    
    # Log model to MLflow
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="SMS_Spam_RandomForest"
    )
    
    print(f"\n✓ Model registered as 'SMS_Spam_RandomForest'")
    print(f"✓ Test AUCPR: {test_metrics['aucpr']:.4f}")
    
    rf_run_id = run.info.run_id
    rf_aucpr = test_metrics['aucpr']

## 9. Model Comparison - AUCPR Summary

In [ ]:
# Retrieve all runs from the experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Filter and display AUCPR metrics
print("\n" + "="*70)
print("MODEL COMPARISON - AUCPR (Area Under Precision-Recall Curve)")
print("="*70)

# Sort by test AUCPR (descending)
runs_sorted = runs.sort_values('metrics.test_aucpr', ascending=False)

print(f"\n{'Model':<30} {'Test AUCPR':<15} {'Val AUCPR':<15} {'Train AUCPR':<15}")
print("-"*70)

for idx, row in runs_sorted.iterrows():
    model_name = row['tags.mlflow.runName']
    test_aucpr = row['metrics.test_aucpr']
    val_aucpr = row['metrics.val_aucpr']
    train_aucpr = row['metrics.train_aucpr']
    
    print(f"{model_name:<30} {test_aucpr:<15.4f} {val_aucpr:<15.4f} {train_aucpr:<15.4f}")

print("\n" + "="*70)

# Find best model
best_model_row = runs_sorted.iloc[0]
best_model_name = best_model_row['tags.mlflow.runName']
best_aucpr = best_model_row['metrics.test_aucpr']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Test AUCPR: {best_aucpr:.4f}")
print(f"   Run ID: {best_model_row['run_id']}")
print("="*70)

## 10. Retrieve and Print Individual Model AUCPR

In [ ]:
# Retrieve AUCPR for each model from MLflow
print("\n" + "="*70)
print("INDIVIDUAL MODEL AUCPR RETRIEVAL FROM MLFLOW")
print("="*70)

model_names = [
    "Logistic_Regression",
    "Naive_Bayes",
    "Random_Forest"
]

for model_name in model_names:
    # Search for runs with this name
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        filter_string=f"tags.mlflow.runName = '{model_name}'",
        order_by=["start_time DESC"],
        max_results=1
    )
    
    if len(runs) > 0:
        run = runs.iloc[0]
        run_id = run['run_id']
        test_aucpr = run['metrics.test_aucpr']
        val_aucpr = run['metrics.val_aucpr']
        train_aucpr = run['metrics.train_aucpr']
        
        print(f"\n📊 {model_name}")
        print(f"   Run ID: {run_id}")
        print(f"   Test AUCPR:       {test_aucpr:.4f}")
        print(f"   Validation AUCPR: {val_aucpr:.4f}")
        print(f"   Training AUCPR:   {train_aucpr:.4f}")
        print(f"   {'-'*50}")

print("\n" + "="*70)

## 11. View Registered Models

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

print("\n" + "="*70)
print("REGISTERED MODELS IN MLFLOW MODEL REGISTRY")
print("="*70)

# List all registered models
registered_models = client.search_registered_models()

for rm in registered_models:
    if rm.name.startswith("SMS_Spam_"):
        print(f"\nModel Name: {rm.name}")
        print(f"Description: {rm.description if rm.description else 'N/A'}")
        print(f"Latest Version: {rm.latest_versions[0].version if rm.latest_versions else 'N/A'}")
        print(f"Last Updated: {rm.last_updated_timestamp}")
        print("-"*70)

print("\n✓ All models successfully registered and tracked with MLflow!")